## Librerias
Es necesario instalar selenium, requests no puede acceder al cuerpo del html de la página, y se necesita time para que la página no ponga el mensaje de "confirma que no eres un robot"

In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver                  # hay que instalar este paquete
import numpy as np
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
def getAndParseURL(url):
    driver = webdriver.Firefox()#options=opts)    # si no usan firefox cambian esta linea por el nombre de su navegador
    driver.get(url)

    page_source = driver.page_source
    soup = bs(page_source,features="html.parser")
    time.sleep(5)
    driver.close()
    return soup

In [3]:
def encontrar_ano(string):
    anos = [f'20{i}{j}' for j in range(10) for i in range(3)]
    anos2 = [f'199{j}' for j in range(10)]
    anos += anos2
    for ano in anos:
        if ano in string:
            return ano
    return np.nan

In [4]:
def get_marca(string):
    marcas = ['kia', 'hyundai', 'honda', 'chery', 'mg', 'mazda', 'suzuki']
    for marca in marcas:
        if marca in string.lower():
            return marca
    return np.nan

def get_modelo(string):
    modelos = ['accent', 'rio-5', 'wr-v', 'cr-v', '3', 'tiggo-2', 'baleno', 'xv', 'swift', 'grand-i10', '2']
    for modelo in modelos:
        if modelo in string.lower():
            return modelo
    return np.nan

In [5]:
# para obtener todas las páginas
def get_pag_sgte(soup):
    next_page = soup.find('a', {'data-webm-clickvalue':'sv-bpg-next'})
    #print(next_page)
    try:
        url = next_page.get('href')
        return url
    except:
        print('ultima página')
        return False

In [13]:
def get_info_autos(soup):
    datos = []
    publicaciones = soup.find_all('div', class_='card-body')
    for pub in publicaciones:
        try:
            titulo = pub.find('a', class_='js-encode-search')
            marca = get_marca(titulo.text.strip('\n').replace(' ', ''))
            modelo = get_modelo(titulo.text.strip('\n').replace(' ', ''))
            ano = encontrar_ano(titulo.text.strip('\n'))
        except:
            marca = np.nan
            modelo = np.nan
            ano = np.nan
        try:
            kilometraje = pub.find('li', {'class':'key-details__value', 'data-type':'Odometer'}).text.strip('\n')
        except:
            kilometraje = np.nan
        try:
            transmision = pub.find('li', {'class':'key-details__value', 'data-type':'Transmission'}).text.strip('\n')
        except: 
            transmision = np.nan
        try:
            precio = pub.find('div', class_='price').a.text
            precio = precio.replace('$', '').replace(',', '').replace('CLP', '').replace(' ', '')
            #precio = precio.replace(',', '')
        except:
            precio = np.nan
        datos.append((marca, modelo, ano, kilometraje, transmision, precio))

    return datos

In [51]:
def get_info_autos2(soup: bs, marca: str):
    datos = []
    publicaciones = soup.find_all('div', {'class':'listing-item card showcase'})#, 'data-webm-make':marca.capitalize()})
    for pub in publicaciones:
        #print('\n----------------------------------------------------------------------\n')
        cuerpo = pub.find('div', class_='card-body')
        try:
            titulo = pub.find('a', {'class':'js-encode-search', 'data-webm-clickvalue': 'sv-title'}).text.strip('\n')
            #print(titulo)
            marca = get_marca(titulo.replace(' ', ''))
            modelo = get_modelo(titulo.replace(' ', ''))
            ano = encontrar_ano(titulo)
        except:
            marca = np.nan
            modelo = np.nan
            ano = np.nan
        try:
            kilometraje = pub.find('li', {'class':'key-details__value', 'data-type':'Odometer'}).text.strip('\n')
        except:
            kilometraje = np.nan
        try:
            transmision = pub.find('li', {'class':'key-details__value', 'data-type':'Transmission'}).text.strip('\n')
        except: 
            transmision = np.nan
        #precio
        try:
            precio = pub.find('div', class_='price').a.text
            precio = precio.replace('$', '').replace(',', '').replace('CLP', '').replace(' ', '')
            #precio = precio.replace(',', '')
        except:
            precio = np.nan
        try:
            tipo_vendedor = pub.find('div', class_='seller-type').text
        except:
            tipo_vendedor = np.nan
        try:
            region = pub.find('div', class_='seller-location d-flex').text
        except:
            region = np.nan
        pie_de_pagina = pub.find('div', class_='card-footer')
        try:
            tipo_vendedor = pub.find('div', class_='seller-type').text.strip('\n')
        except:
            tipo_vendedor = np.nan
        try:
            region = pub.find('div', class_='seller-location d-flex').text
        except:
            region = np.nan

        datos.append((marca, modelo, ano, kilometraje, transmision, tipo_vendedor, region, precio))

    return datos

In [52]:
def contruir_df_auto(marca, modelo):
    df = pd.DataFrame(columns=['Marca', 'Modelo', 'Año', 'Kilometraje', 'Transmision', 'Tipo de Vendedor', 'Región', 'Precio'])
    i = 1
    url_og = f'https://www.chileautos.cl/vehiculos/autos-veh%C3%ADculo/{marca}/{modelo}/'
    seguir = True

    while seguir:   # mientras haya una sgte página
        if i == 1:
            soup = getAndParseURL(url_og) 
        else:
            soup = getAndParseURL(url)
        datos_pag = get_info_autos2(soup, marca)
        for publicacion in datos_pag:
            marca_auto, modelo_auto, ano, kilometraje, transmision, tipo_vendedor, region, precio = publicacion
            df = df.append({'Marca':marca_auto, 'Modelo':modelo_auto, 'Año':ano, 'Kilometraje':kilometraje, 'Transmision':transmision, 'Tipo de Vendedor': tipo_vendedor, 'Región': region, 'Precio': precio}, \
                ignore_index=True)
            #serie = pd.Series({'Marca':[marca_auto], 'Modelo':[modelo_auto], 'Año':[ano], 'Kilometraje':[kilometraje], 'Transmision':[transmision], 'Precio': [precio]})
            #df = pd.concat([df, serie], axis=0)
        i += len(datos_pag) + 1
        if i >= 50:
            return df
        url = get_pag_sgte(soup)
        if url != False:
            try:
                url = 'https://www.chileautos.cl' + url
            except:
                seguir = False
        else:
            seguir = False
        time.sleep(10)
    
    return df

In [47]:
df_wrv = contruir_df_auto('honda', 'wr-v')

In [48]:
df_wrv.to_csv('df_honda_wrv.csv', index=False, encoding='utf-8')
df_wrv

,Marca,Modelo,Año,Kilometraje,Transmision,Tipo de Vendedor,Región,Precio
0,honda,wr-v,2018,"76,022 km",Manual,Automotora Usado,Metropolitana de Santiago,10990000
1,honda,wr-v,2022,"71,000 km",Manual,Automotora Usado,Metropolitana de Santiago,14880000
2,honda,wr-v,2022,NaN,Manual,Automotora Nuevo,Metropolitana de Santiago,16380000
3,honda,wr-v,2018,"60,500 km",Automática,Automotora Usado,Metropolitana de Santiago,13490000
4,honda,wr-v,2018,"69,000 km",Manual,Automotora Usado,Metropolitana de Santiago,9990000
5,honda,wr-v,2018,"93,696 km",Manual,Automotora Usado,Maule,11490000
6,honda,wr-v,2022,800 km,Automática,Automotora Usado,Metropolitana de Santiago,13990000
7,honda,wr-v,2018,"49,500 km",Manual,Automotora Usado,Los Lagos,11990000
8,honda,wr-v,2022,300 km,Automática,Particular,Valparaíso,18400000
9,honda,wr-v,2017,"32,000 km",Manual,Particular,Metropolitana de Santiago,11200000


In [53]:
df_mazda2 = contruir_df_auto('mazda', '2')

In [54]:
df_mazda2.to_csv('df_mazda2.csv', index=False, encoding='utf-8')
df_mazda2

,Marca,Modelo,Año,Kilometraje,Transmision,Tipo de Vendedor,Región,Precio
0,mazda,2,2016,"75,000 km",Manual,Automotora Usado,Metropolitana de Santiago,11550000
1,mazda,2,2016,"26,130 km",Manual,Automotora Usado,Araucanía,10990000
2,mazda,2,2018,"72,900 km",Manual,Automotora Usado,Metropolitana de Santiago,10590000
3,mazda,2,2019,"26,000 km",Manual,Particular,Metropolitana de Santiago,12900000
4,mazda,2,2011,"130,000 km",Manual,Automotora Usado,Metropolitana de Santiago,5890000
5,mazda,2,2018,"36,000 km",Automática,Automotora Usado,Metropolitana de Santiago,11990000
6,mazda,2,2009,"135,987 km",Automática,Automotora Usado,Metropolitana de Santiago,5980000
7,mazda,2,2019,"25,000 km",Manual,Particular,Valparaíso,12290000
8,mazda,2,2020,"28,000 km",Manual,Particular,Valparaíso,14500000
9,mazda,2,2018,"51,000 km",Automática,Particular,Metropolitana de Santiago,10900000


In [55]:
df_mazda3 = contruir_df_auto('mazda', '3')

In [56]:
def cambiar_a_3(string):
    return 3

df_mazda3['Modelo'] = df_mazda3['Modelo'].apply(cambiar_a_3)

In [57]:
df_mazda3.to_csv('df_mazda3.csv', index=False, encoding='utf-8')
df_mazda3 = df_mazda3.dropna()
df_mazda3

,Marca,Modelo,Año,Kilometraje,Transmision,Tipo de Vendedor,Región,Precio
0,mazda,3,2018,"88,940 km",Automática,Particular,Valparaíso,13800000
1,mazda,3,2014,"87,300 km",Manual,Particular,Metropolitana de Santiago,11500000
2,mazda,3,2018,"40,000 km",Automática,Automotora Usado,Metropolitana de Santiago,15000000
3,mazda,3,2011,"114,000 km",Manual,Automotora Usado,Metropolitana de Santiago,8880000
4,mazda,3,2016,"69,331 km",Automática,Automotora Usado,Metropolitana de Santiago,14990000
5,mazda,3,2020,"33,000 km",Automática,Automotora Usado,Metropolitana de Santiago,18600000
6,mazda,3,2019,"40,000 km",Manual,Automotora Usado,Metropolitana de Santiago,16980000
7,mazda,3,2014,"100,000 km",Manual,Automotora Usado,Metropolitana de Santiago,9600000
8,mazda,3,2018,"66,000 km",Manual,Automotora Usado,Metropolitana de Santiago,14000000
9,mazda,3,2020,"31,000 km",Manual,Automotora Usado,Metropolitana de Santiago,15890000


In [58]:
df_mg3 = contruir_df_auto('mg', '3')

In [59]:
df_mg3.to_csv('df_mg3.csv', index=False, encoding='utf-8')
df_mg3['Modelo'] = df_mg3['Modelo'].apply(cambiar_a_3)  # aplicar esta función en otro archivo, abriendo el csv
df_mg3

,Marca,Modelo,Año,Kilometraje,Transmision,Tipo de Vendedor,Región,Precio
0,mg,3,2021,"12,500 km",Manual,Particular,Metropolitana de Santiago,8000000
1,mg,3,2021,"33,000 km",Manual,Automotora Usado,Metropolitana de Santiago,8480000
2,mg,3,2021,"15,000 km",Manual,Automotora Usado,Metropolitana de Santiago,9480000
3,mg,3,2021,"30,000 km",Automática,Automotora Usado,Metropolitana de Santiago,9690000
4,mg,3,2013,"97,000 km",Manual,Automotora Usado,Metropolitana de Santiago,4990000
5,mg,3,2022,"1,900 km",Manual,Automotora Usado,Metropolitana de Santiago,8990000
6,mg,3,2020,"31,400 km",Automática,Particular,Metropolitana de Santiago,9990000
7,mg,3,2019,"55,000 km",Manual,Particular,Metropolitana de Santiago,7700000
8,mg,3,2018,"53,000 km",Manual,Particular,OHiggins,7000000
9,mg,3,2022,"5,000 km",Manual,Particular,Maule,8990000


In [60]:
df_kia_rio5 = contruir_df_auto('kia', 'rio-5')

In [61]:
df_kia_rio5.to_csv('df_kia_rio5.csv', index=False, encoding='utf-8')
df_kia_rio5

,Marca,Modelo,Año,Kilometraje,Transmision,Tipo de Vendedor,Región,Precio
0,kia,2,2017,"24,500 km",Manual,Automotora Usado,Araucanía,8690000
1,kia,2,2019,"51,179 km",Manual,Automotora Usado,Metropolitana de Santiago,10590000
2,kia,2,2020,"54,000 km",Automática,Automotora Usado,Metropolitana de Santiago,12690000
3,kia,2,2016,"106,031 km",Manual,Automotora Usado,Metropolitana de Santiago,10990000
4,kia,2,2016,"99,000 km",Manual,Automotora Usado,Metropolitana de Santiago,8500000
5,kia,2,2019,"45,000 km",Manual,Automotora Usado,Metropolitana de Santiago,8930000
6,kia,2,2019,"38,804 km",Manual,Automotora Usado,Metropolitana de Santiago,11990000
7,kia,2,2019,"51,574 km",Manual,Automotora Usado,Metropolitana de Santiago,9990000
8,kia,2,2022,"36,000 km",Manual,Automotora Usado,Metropolitana de Santiago,13990000
9,kia,2,2018,"76,000 km",Manual,Automotora Usado,Metropolitana de Santiago,9600000


In [62]:
df_crv = contruir_df_auto('honda', 'cr-v')

KeyboardInterrupt: 

In [11]:
df_crv.to_csv('df_honda_crv.csv', index=False, encoding='utf-8')
df_crv

,Marca,Modelo,Año,Kilometraje,Transmision,Precio
0,honda,cr-v,2017,"106,000 km",Automática,16790000
1,honda,cr-v,2015,"107,000 km",Automática,14500000
2,honda,cr-v,2017,"76,000 km",Automática,21580000
3,NaN,NaN,NaN,NaN,NaN,NaN
4,honda,cr-v,2023,NaN,Automática,33580000
...,...,...,...,...,...,...
143,honda,cr-v,2012,"93,000 km",Automática,12400000
144,honda,cr-v,2015,"122,685 km",Automática,13800000
145,honda,cr-v,2007,242 km,Automática,7000000
146,honda,cr-v,2014,"82,000 km",Automática,14590000


In [63]:
df_tiggo2 = contruir_df_auto('chery', 'tiggo-2')

In [64]:
df_tiggo2.to_csv('df_tiggo2.csv', index=False, encoding='utf-8')
df_tiggo2   #

,Marca,Modelo,Año,Kilometraje,Transmision,Tipo de Vendedor,Región,Precio
0,chery,2,2018,"64,273 km",Manual,Automotora Usado,Bío Bío,8290000
1,chery,2,2020,"50,177 km",Manual,Automotora Usado,Metropolitana de Santiago,8490000
2,chery,2,2021,"12,500 km",Automática,Automotora Usado,Metropolitana de Santiago,10290000
3,chery,2,2022,"8,100 km",Manual,Automotora Usado,OHiggins,9990000
4,chery,2,2022,"4,000 km",Automática,Automotora Usado,Metropolitana de Santiago,10990000
5,chery,2,2021,"20,000 km",Manual,Automotora Usado,Metropolitana de Santiago,9190000
6,chery,2,2020,"20,000 km",Manual,Automotora Usado,Metropolitana de Santiago,8990000
7,chery,2,2017,"58,000 km",Manual,Automotora Usado,Metropolitana de Santiago,7700000
8,chery,2,2018,"60,000 km",Manual,Automotora Usado,Metropolitana de Santiago,8700000
9,chery,2,2021,"50,500 km",Manual,Automotora Usado,OHiggins,8990000


In [65]:
df_baleno = contruir_df_auto('suzuki', 'baleno')

In [66]:
df_baleno.to_csv('df_baleno.csv', index=False, encoding='utf-8')
df_baleno

,Marca,Modelo,Año,Kilometraje,Transmision,Tipo de Vendedor,Región,Precio
0,suzuki,baleno,2021,"27,000 km",Manual,Particular,Metropolitana de Santiago,11100000
1,suzuki,baleno,2020,"41,113 km",Manual,Automotora Usado,Araucanía,9490000
2,suzuki,baleno,2022,"29,588 km",Manual,Automotora Usado,Araucanía,10990000
3,suzuki,baleno,2021,"51,622 km",Manual,Automotora Usado,Metropolitana de Santiago,10290000
4,suzuki,baleno,2020,"90,000 km",Manual,Automotora Usado,Metropolitana de Santiago,8990000
5,suzuki,baleno,2018,"45,000 km",Manual,Automotora Usado,Metropolitana de Santiago,9800000
6,suzuki,baleno,2019,"11,000 km",Manual,Automotora Usado,Metropolitana de Santiago,10990000
7,suzuki,baleno,2020,"29,000 km",Manual,Automotora Usado,Metropolitana de Santiago,10500000
8,suzuki,baleno,2022,"17,500 km",Manual,Automotora Usado,Metropolitana de Santiago,11790000
9,suzuki,baleno,2020,"39,061 km",Automática,Automotora Usado,Bío Bío,13290000


In [67]:
df_grandi10  = contruir_df_auto('hyundai', 'grand-i10')

In [68]:
df_grandi10.to_csv('df_gradni10.csv', index=False, encoding='utf-8')
df_grandi10

,Marca,Modelo,Año,Kilometraje,Transmision,Tipo de Vendedor,Región,Precio
0,hyundai,2,2021,"51,150 km",Manual,Automotora Usado,Metropolitana de Santiago,8990000
1,hyundai,2,2022,NaN,Manual,Automotora Nuevo,Metropolitana de Santiago,12390000
2,hyundai,2,2020,"31,909 km",Manual,Automotora Usado,Metropolitana de Santiago,7990000
3,hyundai,2,2021,"41,784 km",Manual,Automotora Usado,Metropolitana de Santiago,8690000
4,hyundai,2,2020,"35,030 km",Manual,Automotora Usado,Metropolitana de Santiago,7990000
5,hyundai,2,2021,"19,489 km",Manual,Automotora Usado,Metropolitana de Santiago,8990000
6,hyundai,2,2021,"10,059 km",Manual,Automotora Usado,Bío Bío,9590000
7,hyundai,2,2022,"26,000 km",Manual,Automotora Usado,Valparaíso,10290000
8,hyundai,2,2021,"17,500 km",Manual,Automotora Usado,Metropolitana de Santiago,9250000
9,hyundai,2,2016,"40,000 km",Automática,Automotora Usado,Metropolitana de Santiago,7800000
